In [1]:
# Data Wrangling
# MBA em Data Science e Analytics USP/ESALQ

In [2]:
#%% Atividade nº 3 - Dataset Comissão de Valores Mobiliários (CVM)

# O dataset contém informações financeiras de companhias abertas brasileiras
# Fonte: https://dados.cvm.gov.br/dataset/cia_aberta-doc-dfp

# O objetivo é analisar a variação anual nas vendas e no lucro das empresas

In [3]:
#%% Carregando os pacotes

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Matplotlib created a temporary cache directory at /tmp/matplotlib-mg6qb6vx because the default path (/home/coder/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [4]:
#%% Importando os bancos de dados

# Os arquivos têm um encoding específico
# É possível ajustar já na importação do .csv

dados_cvm = pd.read_csv('(3.2)_CVM_Resultado.csv', 
                        sep=';',
                        encoding='latin1')

dados_cadastro = pd.read_csv('(3.3)_CVM_Dados_Cadastrais.csv', 
                        sep=';',
                        encoding='latin1')

In [6]:
#%% Registros únicos das contas

contas = dados_cvm['DS_CONTA'].unique()

print(contas)

['Receitas de Intermediação Financeira' 'Receita de Juros'
 'Despesas de Intermediação Financeira' ... 'Venda de mercadorias'
 'Serviços prestados' 'Participação do Empregados']


In [7]:
#%% Filtrar as observações de interesse

# Em análise detalhada, identificou-se pelo código das contas (CD_CONTA)
# A receita principal da empresa é 3.01 e lucro/prejuízo líquido é 3.11
dados_sel = dados_cvm.query('CD_CONTA == "3.01" | CD_CONTA == "3.11"')

In [8]:
#%% Vamos ajustar a variável de data (que está como texto)

data = pd.to_datetime(dados_sel['DT_FIM_EXERC']).dt.year
dados_sel.insert(5, 'ANO', data)

dados_sel.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1302 entries, 0 to 23469
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CNPJ_CIA       1302 non-null   object 
 1   DT_REFER       1302 non-null   object 
 2   VERSAO         1302 non-null   int64  
 3   DENOM_CIA      1302 non-null   object 
 4   CD_CVM         1302 non-null   int64  
 5   ANO            1302 non-null   int32  
 6   GRUPO_DFP      1302 non-null   object 
 7   MOEDA          1302 non-null   object 
 8   ESCALA_MOEDA   1302 non-null   object 
 9   ORDEM_EXERC    1302 non-null   object 
 10  DT_INI_EXERC   1302 non-null   object 
 11  DT_FIM_EXERC   1302 non-null   object 
 12  CD_CONTA       1302 non-null   object 
 13  DS_CONTA       1302 non-null   object 
 14  VL_CONTA       1302 non-null   float64
 15  ST_CONTA_FIXA  1302 non-null   object 
dtypes: float64(1), int32(1), int64(2), object(12)
memory usage: 167.8+ KB


In [9]:
#%% Organizar as observações

# Temos informações de 2021 e 2022, vamos colocá-las juntas para cada empresa
# Para melhor organização, vamos separar as contas de receitas e lucros
dados_sel = dados_sel.sort_values(by=['CD_CONTA', 'CD_CVM'], ascending=True)

In [10]:
#%% Análise de duplicidades de observações

contagem = dados_sel.groupby(['CD_CVM', 'CD_CONTA'])['VL_CONTA'].count()

# Há um resíduo no dataset, a empresa CD_CVM = 26077 tem duplicidades

In [11]:
#%% Exclusão do resíduo

# Em análise adicional, verificou-se que há "versões" de relatórios
# Vamos manter a última versão disponibilizada (VERSAO = 3)
dados_sel.query('~(CD_CVM == 26077 & VERSAO == 1)', inplace=True)

# A inversão do critério de filtro foi feita por meio do ~

In [12]:
#%% Vamos adicionar os setores das empresas para fazer análises específicas

# Ajustando a base dos dados cadastrais
cadastrais = dados_cadastro[['CD_CVM', 'SETOR_ATIV']].copy()
cadastrais = cadastrais[cadastrais['SETOR_ATIV'].notnull()] # elimina missings

# Vamos manter apenas registros únicos (evitando duplicidade no merge)
cadastrais.drop_duplicates(inplace=True)

# Realizando o merge
dados_sel = pd.merge(dados_sel, cadastrais, 
                     how="left", 
                     on="CD_CVM")

In [13]:
#%% Limpeza: vamos selecionar apenas as variáveis de interesse

dados_sel = dados_sel[['CD_CVM',
                       'DENOM_CIA', 
                       'SETOR_ATIV', 
                       'CD_CONTA', 
                       'ANO', 
                       'VL_CONTA']]

# Note que já reposicionamos na ordem desejada

In [14]:
#%% Para facilitar a leitura das informações, vamos substituir os labels

labels = {'3.01': 'Receita',
          '3.11': 'Resultado'}

dados_sel = dados_sel.assign(CD_CONTA = dados_sel.CD_CONTA.map(labels))

In [15]:
#%% Vamos calcular a variação percentual (variável de interesse no estudo)

# Criar uma variável com o valor defasado
dados_sel['VALOR_LAG'] = dados_sel.groupby(['CD_CVM', 'CD_CONTA'])['VL_CONTA'].shift(1)

# Criando uma variável com o resultado da variação
dados_sel['VARIACAO'] = ((dados_sel['VL_CONTA'] - dados_sel['VALOR_LAG']) / dados_sel['VALOR_LAG'])

# Vamos ajustar o arredondamento para melhor visualização
dados_sel['VARIACAO'] = round(dados_sel['VARIACAO'], 3)

In [16]:
#%% Estatísticas descritivas da VARIAÇÃO

# Existem valores muito extremos que influenciam as descritivas

# Vamos limpar as observações
dados_sel = dados_sel[~ dados_sel['VARIACAO'].isin([np.nan, np.inf, -np.inf])]

# Tabela de estatísticas descritivas da VARIAÇÃO
dados_sel['VARIACAO'].describe()

count    640.000000
mean      -0.072502
std        6.245403
min     -111.283000
25%       -0.168750
50%        0.146000
75%        0.424750
max       44.180000
Name: VARIACAO, dtype: float64

In [17]:
#%% Vamos excluir grandes variações

# Existem muitos valores extremos na distribuição da variável

# Exemplo: vamos excluir variações maiores do que 200% e menores do que -200%
# São indícios de variações significativas nos fundamentos da empresa

dados_sel = dados_sel[dados_sel['VARIACAO'].between(-2, 2, inclusive='both')]

In [18]:
#%% Novas estatísticas descritivas

dados_sel['VARIACAO'].describe()

count    592.000000
mean       0.121939
std        0.573287
min       -1.845000
25%       -0.127250
50%        0.136500
75%        0.374250
max        1.964000
Name: VARIACAO, dtype: float64

In [19]:
#%% Informações mais detalhadas por tipo de conta (receita e resultado)

dados_sel.groupby(['CD_CONTA'])['VARIACAO'].describe().T

CD_CONTA,Receita,Resultado
count,316.000000,276.000000
mean,0.260741,-0.036978
std,0.406167,0.685466
min,-1.630000,-1.845000
25%,0.051500,-0.524000
50%,0.208000,-0.096000
75%,0.398250,0.326250
max,1.860000,1.964000


In [20]:
#%% Informações mais detalhadas por setor

# Por setor
desc_setor = dados_sel.groupby(['CD_CONTA','SETOR_ATIV']).agg({'VARIACAO':'mean'})
desc_setor = desc_setor.reset_index()

# Os números setoriais indicam que existem análises mais específicas a fazer
# Por exemplo, alguns setores podem ter poucas observações (média com viés)

# Contagem de informações por setor
# Critério escolhido: vamos manter apenas setores com no mínimo 6 observações
n_setor = dados_sel[['SETOR_ATIV', 'VARIACAO']].groupby('SETOR_ATIV').count()

n_setor_seleciona = (n_setor
                     .query('VARIACAO >= 6')
                     .rename(columns={'VARIACAO':'CONTAGEM'})).reset_index()

# Ajuste do banco de dados
desc_setor = (desc_setor
.merge(n_setor_seleciona, how = 'left', on = 'SETOR_ATIV')
.query('~CONTAGEM.isna()'))

# Visualizando graficamente
plt.figure(figsize=(18,12), dpi = 600)
sns.barplot(data=desc_setor, y='SETOR_ATIV', x='VARIACAO', hue='CD_CONTA')

#%% FIM!

<Axes: xlabel='VARIACAO', ylabel='SETOR_ATIV'>